# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe7541a1e50>
├── 'a' --> tensor([[-0.8480,  0.0480,  0.7417],
│                   [ 0.8259, -1.4979, -0.1696]])
└── 'x' --> <FastTreeValue 0x7fe7541a1eb0>
    └── 'c' --> tensor([[-1.6694, -0.9628, -0.5595, -0.9531],
                        [ 0.1793,  1.7441,  0.0971, -0.4052],
                        [ 0.1178, -0.8629,  1.0390,  1.0302]])

In [4]:
t.a

tensor([[-0.8480,  0.0480,  0.7417],
        [ 0.8259, -1.4979, -0.1696]])

In [5]:
%timeit t.a

59.2 ns ± 0.0296 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe7541a1e50>
├── 'a' --> tensor([[-1.0564, -1.5127,  0.1632],
│                   [-0.6388, -0.3427, -0.1764]])
└── 'x' --> <FastTreeValue 0x7fe7541a1eb0>
    └── 'c' --> tensor([[-1.6694, -0.9628, -0.5595, -0.9531],
                        [ 0.1793,  1.7441,  0.0971, -0.4052],
                        [ 0.1178, -0.8629,  1.0390,  1.0302]])

In [7]:
%timeit t.a = new_value

61.4 ns ± 0.0256 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.8480,  0.0480,  0.7417],
               [ 0.8259, -1.4979, -0.1696]]),
    x: Batch(
           c: tensor([[-1.6694, -0.9628, -0.5595, -0.9531],
                      [ 0.1793,  1.7441,  0.0971, -0.4052],
                      [ 0.1178, -0.8629,  1.0390,  1.0302]]),
       ),
)

In [10]:
b.a

tensor([[-0.8480,  0.0480,  0.7417],
        [ 0.8259, -1.4979, -0.1696]])

In [11]:
%timeit b.a

51.2 ns ± 0.0725 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1743,  1.2967, -0.2615],
               [ 0.3134, -1.5596,  1.8734]]),
    x: Batch(
           c: tensor([[-1.6694, -0.9628, -0.5595, -0.9531],
                      [ 0.1793,  1.7441,  0.0971, -0.4052],
                      [ 0.1178, -0.8629,  1.0390,  1.0302]]),
       ),
)

In [13]:
%timeit b.a = new_value

486 ns ± 0.123 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

830 ns ± 0.922 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 19.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 1.4 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

136 µs ± 193 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe6a7e7a7f0>
├── 'a' --> tensor([[[-0.8480,  0.0480,  0.7417],
│                    [ 0.8259, -1.4979, -0.1696]],
│           
│                   [[-0.8480,  0.0480,  0.7417],
│                    [ 0.8259, -1.4979, -0.1696]],
│           
│                   [[-0.8480,  0.0480,  0.7417],
│                    [ 0.8259, -1.4979, -0.1696]],
│           
│                   [[-0.8480,  0.0480,  0.7417],
│                    [ 0.8259, -1.4979, -0.1696]],
│           
│                   [[-0.8480,  0.0480,  0.7417],
│                    [ 0.8259, -1.4979, -0.1696]],
│           
│                   [[-0.8480,  0.0480,  0.7417],
│                    [ 0.8259, -1.4979, -0.1696]],
│           
│                   [[-0.8480,  0.0480,  0.7417],
│                    [ 0.8259, -1.4979, -0.1696]],
│           
│                   [[-0.8480,  0.0480,  0.7417],
│                    [ 0.8259, -1.4979, -0.1696]]])
└── 'x' --> <FastTreeValue 0x7fe6a7e7a8b0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 75.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe6aeab2a00>
├── 'a' --> tensor([[-0.8480,  0.0480,  0.7417],
│                   [ 0.8259, -1.4979, -0.1696],
│                   [-0.8480,  0.0480,  0.7417],
│                   [ 0.8259, -1.4979, -0.1696],
│                   [-0.8480,  0.0480,  0.7417],
│                   [ 0.8259, -1.4979, -0.1696],
│                   [-0.8480,  0.0480,  0.7417],
│                   [ 0.8259, -1.4979, -0.1696],
│                   [-0.8480,  0.0480,  0.7417],
│                   [ 0.8259, -1.4979, -0.1696],
│                   [-0.8480,  0.0480,  0.7417],
│                   [ 0.8259, -1.4979, -0.1696],
│                   [-0.8480,  0.0480,  0.7417],
│                   [ 0.8259, -1.4979, -0.1696],
│                   [-0.8480,  0.0480,  0.7417],
│                   [ 0.8259, -1.4979, -0.1696]])
└── 'x' --> <FastTreeValue 0x7fe6aeab2e20>
    └── 'c' --> tensor([[-1.6694, -0.9628, -0.5595, -0.9531],
                        [ 0.1793,  1.7441,  0.0971, -0.4052],
                 

In [23]:
%timeit t_cat(trees)

30.9 µs ± 58.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.9 µs ± 141 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.8480,  0.0480,  0.7417],
                [ 0.8259, -1.4979, -0.1696]],
       
               [[-0.8480,  0.0480,  0.7417],
                [ 0.8259, -1.4979, -0.1696]],
       
               [[-0.8480,  0.0480,  0.7417],
                [ 0.8259, -1.4979, -0.1696]],
       
               [[-0.8480,  0.0480,  0.7417],
                [ 0.8259, -1.4979, -0.1696]],
       
               [[-0.8480,  0.0480,  0.7417],
                [ 0.8259, -1.4979, -0.1696]],
       
               [[-0.8480,  0.0480,  0.7417],
                [ 0.8259, -1.4979, -0.1696]],
       
               [[-0.8480,  0.0480,  0.7417],
                [ 0.8259, -1.4979, -0.1696]],
       
               [[-0.8480,  0.0480,  0.7417],
                [ 0.8259, -1.4979, -0.1696]]]),
    x: Batch(
           c: tensor([[[-1.6694, -0.9628, -0.5595, -0.9531],
                       [ 0.1793,  1.7441,  0.0971, -0.4052],
                       [ 0.1178, -0.8629,  1.0390,  1.0302]],
         

In [26]:
%timeit Batch.stack(batches)

77.7 µs ± 179 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.8480,  0.0480,  0.7417],
               [ 0.8259, -1.4979, -0.1696],
               [-0.8480,  0.0480,  0.7417],
               [ 0.8259, -1.4979, -0.1696],
               [-0.8480,  0.0480,  0.7417],
               [ 0.8259, -1.4979, -0.1696],
               [-0.8480,  0.0480,  0.7417],
               [ 0.8259, -1.4979, -0.1696],
               [-0.8480,  0.0480,  0.7417],
               [ 0.8259, -1.4979, -0.1696],
               [-0.8480,  0.0480,  0.7417],
               [ 0.8259, -1.4979, -0.1696],
               [-0.8480,  0.0480,  0.7417],
               [ 0.8259, -1.4979, -0.1696],
               [-0.8480,  0.0480,  0.7417],
               [ 0.8259, -1.4979, -0.1696]]),
    x: Batch(
           c: tensor([[-1.6694, -0.9628, -0.5595, -0.9531],
                      [ 0.1793,  1.7441,  0.0971, -0.4052],
                      [ 0.1178, -0.8629,  1.0390,  1.0302],
                      [-1.6694, -0.9628, -0.5595, -0.9531],
                      [ 0.1793,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 138 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

313 µs ± 2.18 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
